In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import keras
import tensorflow as tf
import PIL

from tensorflow.keras import layers
from tensorflow.keras import Sequential

import re

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
ds = tf.data.Dataset.from_tensor_slices((train_data["example_path"], train_data["label"]))

In [4]:
DATASET_SIZE = train_data.shape[0]

In [5]:
train_size = int(0.8 * DATASET_SIZE)

In [6]:
ds.shuffle(1000)
train = ds.take(train_size)
val = ds.skip(train_size)

In [7]:
def preprocess(image, label):
    img = tf.io.read_file(image)
    img = tf.io.decode_png(img, channels=3,dtype=tf.dtypes.uint16)
    img = tf.cast(img,dtype=tf.float64)
    resized_image = tf.image.resize(img, [332, 332])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [8]:
train = train.map(preprocess).prefetch(tf.data.AUTOTUNE)
val = val.map(preprocess).prefetch(tf.data.AUTOTUNE)

In [9]:
data_augmentation = keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",
                      input_shape=(332,
                                  332,
                                  3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)


In [10]:
def generate_images(df):
    df_batch_1 = df.batch(1, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    for image, label in df_batch_1:
        if(label == 1):
            conv_im = data_augmentation(image)
            ds = tf.data.Dataset.from_tensors((conv_im[0], label))
            df = df.concatenate(ds)
    return df

In [125]:
train = generate_images(train)
val = generate_images(val)

In [11]:
batch_size = 32
train = train.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val = val.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [14]:
num_classes = 3

model = Sequential([
  layers.Rescaling(1./255, input_shape=(332, 332, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 332, 332, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 332, 332, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 166, 166, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 166, 166, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 83, 83, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 83, 83, 64)       

In [ ]:
epochs=10
history = model.fit(
  train,
  validation_data=val,
  epochs=epochs
)

Epoch 1/10
42/42 [==============================] - 54s 1s/step - loss: 1.0318 - accuracy: 0.5037 - val_loss: 0.9250 - val_accuracy: 0.5594
Epoch 2/10
42/42 [==============================] - 55s 1s/step - loss: 0.8253 - accuracy: 0.6146 - val_loss: 0.8726 - val_accuracy: 0.6250
Epoch 3/10
42/42 [==============================] - 57s 1s/step - loss: 0.6739 - accuracy: 0.6949 - val_loss: 0.9364 - val_accuracy: 0.5375
Epoch 4/10
42/42 [==============================] - 57s 1s/step - loss: 0.5307 - accuracy: 0.7664 - val_loss: 0.9053 - val_accuracy: 0.5906
Epoch 5/10
42/42 [==============================] - 56s 1s/step - loss: 0.4031 - accuracy: 0.8296 - val_loss: 1.1870 - val_accuracy: 0.6000
Epoch 6/10
42/42 [==============================] - 58s 1s/step - loss: 0.2349 - accuracy: 0.9204 - val_loss: 1.4338 - val_accuracy: 0.6000
Epoch 7/10
 3/42 [=>............................] - ETA: 52s - loss: 0.1530 - accuracy: 0.9479